In [103]:
import numpy as np
import pandas as pd
import datetime
from sodapy import Socrata
import urllib
import dill
import future
from censusgeocode import CensusGeocode
import ast
import time
import math

In [710]:
with open('pluto.pkl', 'rb') as file:
    datastruct = dill.load(file)

In [378]:
allcalls2 = dict(allcalls)
allcalls2[2004].head()

,full_address,APPLIANCE,CONSTRUCTION,ELECTRIC,GENERAL CONSTRUCTION,HEATING,NONCONST,PAINT - PLASTER,PLUMBING,Borough,...,MajPrpDate,LandmkDate,BaseMpDate,MASDate,PoliDate,PLUTOMapID,latitude,longitude,incident_zip,incident_address
0,"1 74 STREET, 11209",0.0,0.0,0.0,0.0,8.0,0.0,1.0,2.0,BK,...,Oct-04,Nov-04,Aug-04,Nov-04,Jul-04,1.0,40.634316253654525,-74.03659828262641,11209,1 74 STREET
11,"1 ADLER PLACE, 10475",0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.87147458916674,-73.82818296460148,10475,1 ADLER PLACE
13,"1 ADRIAN AVENUE, 10463",1.0,0.0,4.0,1.0,5.0,0.0,1.0,6.0,MN,...,10/2004,11/2004,08/2004,11/2004,07/2004,1.0,40.8758976843239,-73.91246765124495,10463,1 ADRIAN AVENUE
31,"1 ARLINGTON PLACE, 11216",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,BK,...,Oct-04,Nov-04,Aug-04,Nov-04,Jul-04,1.0,40.68152473461247,-73.95129032944702,11216,1 ARLINGTON PLACE
32,"1 AUDUBON AVENUE, 10032",0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,MN,...,10/2004,11/2004,08/2004,11/2004,07/2004,1.0,40.83883689995949,-73.93958873444733,10032,1 AUDUBON AVENUE


In [5]:
#Program for scraping desired 311 call data====================================
client = Socrata("data.cityofnewyork.us", 
                 "Ntm48wnKDuTGKcR9PppjuQn2l", 
                 username="sdg2145@columbia.edu", 
                 password="Gracie2Mia!")

In [6]:
cg =  CensusGeocode(benchmark='Public_AR_Census2010', vintage='Census2000_Census2010')#2000 Census Tracts
cg2 = CensusGeocode(benchmark='Public_AR_Census2010', vintage='Census2010_Census2010') #2010 Census Tracts

In [7]:
def coord(col1, col2):
    try:
        print("Still Working")
        loc = cg.coordinates(x = col1, y = col2)
        loc = str(loc)
        loc = loc.replace("[", "")
        loc = loc.replace("]", "")
        loc = ast.literal_eval(loc)
        return(loc['Census Tracts']['TRACT'])
    except:
        print("Not Found")
        return(float("NaN"))

In [8]:
def coord10(col1, col2):
    try:
        print("Still Working")
        loc = cg2.coordinates(x = col1, y = col2)
        loc = str(loc)
        loc = loc.replace("[", "")
        loc = loc.replace("]", "")
        loc = ast.literal_eval(loc)
        return(loc['Census Tracts']['TRACT'])
    except:
        print("Not Found")
        return(float("NaN"))

In [677]:
#Scrape HPD Calls and Sum by Address From 2004-2016

    
for year in [2014,2016]:
        print(year)
        year1 = "'" + str(year) + '-01-01T00:01:00.000' + "'"
        year2 = "'" + str(year) + '-12-31T23:59:00.000' + "'"
        
        cd = "created_date between " + year1 + " and " + year2 +  \
             " AND agency = 'HPD'"
        
        df = pd.DataFrame(client.get("fhrw-4uyv", 
                                     content_type = "csv", 
                                     limit = 100000000000, 
                                     where = cd))
        
        df.columns = df.iloc[0]
        df = df.reindex(df.index.drop(0))
        
        df['full_address'] = df['incident_address'] + ", " \
                             + df['incident_zip']
        df2 = df[['latitude', 'longitude', 'full_address', 'incident_zip', 'incident_address']]
        yr = str(year)[2:4]
        df = df.groupby(['full_address', 'complaint_type']).size()
        df = df.unstack('complaint_type').fillna(0).stack().reset_index()
        df.columns.values[2] = "Calls"
        df = df.pivot(values = 'Calls', 
                      columns = 'complaint_type', 
                      index = 'full_address').reset_index()
        yr = str(year)[2:4]
        
        if(yr == '08'):
            yr == '07'
            
        df = pd.merge(df, pluto_dict[yr], how = 'left', on = 'full_address')
        df = pd.merge(df, df2, how = 'left', on = 'full_address')
        df = df.drop_duplicates(subset = 'full_address', keep = 'first')
        
        allcalls[year] = df

#2004-2010 
  

2014
2016


In [721]:
allcalls[2014].isnull().sum()

full_address                0
AGENCY                      0
APPLIANCE                   0
CONSTRUCTION                0
DOOR/WINDOW                 0
ELECTRIC                    0
ELEVATOR                    0
FLOORING/STAIRS             0
GENERAL                     0
GENERAL CONSTRUCTION        0
HEAT/HOT WATER              0
HEATING                     0
NONCONST                    0
OUTSIDE BUILDING            0
PAINT - PLASTER             0
PAINT/PLASTER               0
PLUMBING                    0
SAFETY                      0
UNSANITARY CONDITION        0
VACANT APARTMENT            0
WATER LEAK                  0
Borough                  7655
Block                    7655
Lot                      7655
CD                       7655
CT2010                   7655
CB2010                   7658
SchoolDist               7655
Council                  7655
ZipCode                  7655
                        ...  
YearBuilt                7655
BuiltCode               31655
YearAlter1

In [734]:
#FIX 14 and 16

for year in [2014, 2016]:
    df = allcalls[year].iloc[:,0:21]
    yr = str(year)[2:4]
    
    pluto_dict[yr]['Zip'] = pluto_dict[yr]['ZipCode'].str.slice(0,5)
    pluto_dict[yr]['full_address'] = pluto_dict[yr]['Address'].str.strip() +\
                                      ", " + pluto_dict[yr]['Zip']
       
    df = pd.merge(df, pluto_dict[yr] , how = 'left', on = 'full_address')
    df = df.drop_duplicates(subset = 'full_address', keep = 'first')

    #Fix Census Tracts    
    df['CT2010'] = df['CT2010'].astype(str)
    df['CT2010'] = df.apply(lambda row: row['CT2010'] if row['CT2010'][-1] != '0' else row['CT2010'][:-2], axis = 1)
    df['CT2010'] = df['CT2010'].replace('nan', float('NaN'))
    
    allcalls[year] = df


In [735]:
#Missing tracts in each year
missing = {}

for key in allcalls.keys():
    if key < 2012:
        missing[key] = allcalls[key]['CT2000'].isnull().sum()
    else:
        missing[key] = allcalls[key]['CT2010'].isnull().sum()

missing

{2004: 4724,
 2005: 6555,
 2006: 14537,
 2007: 8538,
 2008: 6404,
 2009: 7026,
 2010: 6166,
 2011: 6273,
 2012: 6888,
 2013: 11659,
 2014: 7655,
 2015: 15858,
 2016: 16471}

In [252]:
missing_tracts = {}

for year in range(2004,2017):
    print(year)
    if(year < 2012):
        num  =  math.ceil(len(allcalls[year][pd.isnull(allcalls[year]['CT2000'])]) / 1000)
        left =  len(allcalls[year][pd.isnull(allcalls[year]['CT2000'])]) % 1000
    else:
        num  =  math.ceil(len(allcalls[year][pd.isnull(allcalls[year]['CT2010'])]) / 1000)
        left =  len(allcalls[year][pd.isnull(allcalls[year]['CT2010'])]) % 1000

    missing_tracts[year] = {}
    
    for i in range(0, num):
            print(i)
            if(i != num - 1):
                if(year < 2012):
                    df = allcalls[year][pd.isnull(allcalls[year]['CT2000'])][i * 1000: i * 1000 + 1000]
                else:
                    df = allcalls[year][pd.isnull(allcalls[year]['CT2010'])][i * 1000: i * 1000 + 1000]
                    
                df = pd.DataFrame(df[['full_address', 'incident_zip', 'incident_address']])
                df['state'] = "New York"
                df['city'] = "New York"

                df['zip code'] = df['incident_zip']
                df['address']  = df['incident_address']

                df.index = list(range(1,1001))

                cols = ['address','state', 'city', 'zip code']
                df2 = df[cols]
                df['full'] = df['address'] + ", " + df['state'] + ", " + df['city'] + ", " + df['zip code']

                df2.to_csv('batch/batch.csv')

                batch = cg.addressbatch('batch/batch.csv')
                data = pd.DataFrame(batch)
                data = data.rename(columns = {'address':'full'})

                df = pd.merge(df, data, how = 'left', on = 'full')
                missing_tracts[year][i] = df

            else:
                if(year < 2012):
                        df = allcalls[year][pd.isnull(allcalls[year]['CT2000'])][i * 1000: i * 1000 + left]
                else:
                        df = allcalls[year][pd.isnull(allcalls[year]['CT2010'])][i * 1000: i * 1000 + left]

                df = pd.DataFrame(df[['full_address', 'incident_zip', 'incident_address']])
                df['state'] = "New York"
                df['city'] = "New York"

                df['zip code'] = df['incident_zip']
                df['address']  = df['incident_address']

                df.index = list(range(1,left + 1))

                cols = ['address','state', 'city', 'zip code']
                df2 = df[cols]
                df['full'] = df['address'] + ", " + df['state'] + ", " + df['city'] + ", " + df['zip code']

                df2.to_csv('batch/batch.csv')

                batch = cg.addressbatch('batch/batch.csv')
                data = pd.DataFrame(batch)
                data = data.rename(columns = {'address':'full'})

                df = pd.merge(df, data, how = 'left', on = 'full')
                missing_tracts[year][i] = df
        
    

2004
0
1
2
3
4
2005
0
1
2
3
4
5
6
2006
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
2007
0
1
2
3
4
5
6
7
8
2008
0
1
2
3
4
5
6
2009
0
1
2
3
4
5
6
7
2010
0
1
2
3
4
5
6
2011
0
1
2
3
4
5
6
2012
0
1
2
3
4
5
6
2013
0
1
2
3
4
5
6
7
8
9
10
11
2014
0
1
2
3
4
5
6
7
2015
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2016
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [261]:
missing_df = {}

for year in missing_tracts.keys():
    data = pd.DataFrame()
    for key in missing_tracts[year].keys():
        data = data.append(missing_tracts[year][key])
    missing_df[year] = data
        

In [290]:
def fix_tract(tract):
    tract = str(tract)
    
    if(tract[0:2] == '00'):
        tract = tract[2:]
    if(tract[0] == '0'):
        tract = tract[1:]
    if(tract[-1] != '0'):
        tract = tract[:-2] + "." + tract[-2:]
    if(tract[-2:] == '00'):
        tract = tract[:-2]
       
    return tract
    

In [295]:
for year in missing_df.keys():
    missing_df[year]['Tract'] = missing_df[year].apply(lambda row: fix_tract(row['tract']) if pd.notnull(row['tract']) else float('NaN'), axis = 1)

In [752]:
allcalls[2014]['CT2010'].head()

0    NaN
1     59
2     38
3    309
4    251
Name: CT2010, dtype: object

In [738]:
full_merged = {}

for year in allcalls.keys():
    if year < 2012:
        new = missing_df[year][['full_address', 'Tract']]
        full = pd.merge(allcalls[year], new, how = 'left', on = 'full_address')
        full.Tract.fillna(full.CT2000, inplace = True)
        full_merged[year] = full
    else:
        new = missing_df[year][['full_address', 'Tract']]
        full = pd.merge(allcalls[year], new, how = 'left', on = 'full_address')
        full.Tract.fillna(full.CT2010, inplace = True)
        full_merged[year] = full

In [739]:
#Missing tracts in each year
missing = {}

for key in full_merged.keys():
    missing[key] = len(full_merged[key][pd.isnull(full_merged[key]['Tract'])])
missing

{2004: 477,
 2005: 700,
 2006: 731,
 2007: 2373,
 2008: 872,
 2009: 1283,
 2010: 682,
 2011: 734,
 2012: 1213,
 2013: 4710,
 2014: 755,
 2015: 886,
 2016: 908}

In [740]:
for year in full_merged.keys():
    full_merged[year]['Tract'] = full_merged[year]['Tract'].str.strip()

In [608]:
cols = {}

for year in full_merged.keys():
    columns = []
    i = 1
    col = ""
    
    if year != 2006:
        while col != 'Borough':
            col = full_merged[year].columns.values[i]
            if col != 'Borough':
                columns.append(col)
                i+=1

    else: 
        columns = full_merged[year].columns.values[0:9]
        
    cols[year] = columns


{2004: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  'PAINT - PLASTER',
  'PLUMBING'],
 2005: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  'PAINT - PLASTER',
  'PLUMBING'],
 2006: array(['full_address', 'APPLIANCE', 'CONSTRUCTION', 'ELECTRIC',
        'GENERAL CONSTRUCTION', 'HEATING', 'NONCONST', 'PAINT - PLASTER',
        'PLUMBING'], dtype=object),
 2007: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  'PAINT - PLASTER',
  'PLUMBING'],
 2008: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  'PAINT - PLASTER',
  'PLUMBING'],
 2009: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  'PAINT - PLASTER',
  'PLUMBING'],
 2010: ['APPLIANCE',
  'CONSTRUCTION',
  'ELECTRIC',
  'GENERAL CONSTRUCTION',
  'HEATING',
  'NONCONST',
  '

In [741]:
#Census Tract
tract_count = {}
for key in full_merged.keys():
    df = full_merged[key]
    grouped = pd.DataFrame(df.groupby(['Tract'])[cols[key]].sum())
    grouped['Year'] = key
    grouped['Tract'] = grouped.index
    tract_count[key] = grouped


In [742]:
full_merged[2014].head()

,full_address,AGENCY,APPLIANCE,CONSTRUCTION,DOOR/WINDOW,ELECTRIC,ELEVATOR,FLOORING/STAIRS,GENERAL,GENERAL CONSTRUCTION,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version,Zip,Tract
0,"1 23 STREET, 11232",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1 5 AVENUE, 10003",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,103 021,10207.0,NaN,0.0,NaN,1.0,14v1,10003,59
2,"1 74 STREET, 11209",0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,311 011,31803.0,NaN,0.0,NaN,1.0,14v1,11209,38
3,"1 ADRIAN AVENUE, 10463",0.0,0.0,0.0,4.0,5.0,0.0,6.0,10.0,2.0,...,NaN,112 079,10804.0,NaN,0.0,NaN,1.0,14v1,10463,309
4,"1 AUDUBON AVENUE, 10032",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,111N093,10801.0,NaN,0.0,NaN,1.0,14v1,10032,251


In [744]:
#fill in years there are missing/none in a tract

#First create master list
tractlist00 = list(tract_count[2004]['Tract'])

for year in range(2005, 2012):
    
    missing = list(set(tract_count[year]['Tract']) - set(tractlist00))
    tractlist00 = tractlist00 + missing

tractlist10 = list(tract_count[2012]['Tract'])

for year in range(2012, 2017):
    
    missing = list(set(tract_count[year]['Tract']) - set(tractlist10))
    tractlist10 = tractlist10 + missing


In [746]:
for key in range(2004, 2012):
    print(key)
    missing = list(filter(lambda x: x not in set(tract_count[key]['Tract']), tractlist00))
    print(len(missing))
    df = pd.DataFrame(missing)
    df = df.rename(columns = {0:'Tract'})
    df['Year'] = key

    for i in range(0, len(tract_count[key].columns) - 2):
        df[tract_count[key].columns[i]] = 0

    cols = list(df.columns.values[2:]) + list(df.columns.values[0:2])

    df = df[cols]
    tract_count[key] = tract_count[key].append(df, ignore_index = True)

for key in range(2012, 2017):
    print(key)
    missing = list(filter(lambda x: x not in set(tract_count[key]['Tract']), tractlist10))
    df = pd.DataFrame(missing)
    df = df.rename(columns = {0:'Tract'})
    df['Year'] = key

    for i in range(0, len(tract_count[key].columns) - 2):
        df[tract_count[key].columns[i]] = 0

    cols = list(df.columns.values[2:]) + list(df.columns.values[0:2])

    df = df[cols]
    tract_count[key] = tract_count[key].append(df, ignore_index = True)

2004
43
2005
26
2006
58
2007
39
2008
43
2009
38
2010
31
2011
37
2012
2013
2014
2015
2016


In [747]:
for year in range(2005, 2016):
    print(len(tract_count[year]['Tract']))

1271
1271
1271
1271
1271
1271
1271
1402
1402
1402
1402


In [748]:
tract_count[2010].head()

,APPLIANCE,CONSTRUCTION,ELECTRIC,GENERAL CONSTRUCTION,HEATING,NONCONST,PAINT - PLASTER,PLUMBING,Total,Tract,Year
0,1.0,0.0,5.0,18.0,31.0,4.0,32.0,16.0,2116.0,1,2010
1,3.0,0.0,7.0,23.0,40.0,17.0,14.0,17.0,2128.0,10,2010
2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,2014.0,10.01,2010
3,13.0,2.0,33.0,84.0,229.0,86.0,69.0,80.0,2593.0,100,2010
4,5.0,0.0,2.0,17.0,31.0,10.0,8.0,23.0,2101.0,1004,2010


In [749]:
tracts_allyears = pd.DataFrame()

for year in tract_count.keys():
    tracts_allyears = tracts_allyears.append(tract_count[year])

In [750]:
tracts_allyears.head()

,AGENCY,APPLIANCE,Agency Issues,CONSTRUCTION,DOOR/WINDOW,ELECTRIC,ELEVATOR,FLOORING/STAIRS,GENERAL,GENERAL CONSTRUCTION,...,PAINT/PLASTER,PLUMBING,SAFETY,STRUCTURAL,Total,Tract,UNSANITARY CONDITION,VACANT APARTMENT,WATER LEAK,Year
0,NaN,1.0,NaN,0.0,NaN,5.0,NaN,NaN,NaN,18.0,...,NaN,16.0,NaN,NaN,2116.0,1,NaN,NaN,NaN,2010
1,NaN,3.0,NaN,0.0,NaN,7.0,NaN,NaN,NaN,23.0,...,NaN,17.0,NaN,NaN,2128.0,10,NaN,NaN,NaN,2010
2,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,3.0,...,NaN,1.0,NaN,NaN,2014.0,10.01,NaN,NaN,NaN,2010
3,NaN,13.0,NaN,2.0,NaN,33.0,NaN,NaN,NaN,84.0,...,NaN,80.0,NaN,NaN,2593.0,100,NaN,NaN,NaN,2010
4,NaN,5.0,NaN,0.0,NaN,2.0,NaN,NaN,NaN,17.0,...,NaN,23.0,NaN,NaN,2101.0,1004,NaN,NaN,NaN,2010


In [751]:
tracts_allyears.to_csv('tract_panel.csv')

In [743]:
totcalls = {}
for year in tract_count.keys():
    
    col = ""
    columns = []
    i = 1
    while col != 'Tract':
            col = tract_count[year].columns.values[i]
            if col != 'Tract':
                columns.append(col)
                i+=1
    
    tract_count[year]['Total'] = tract_count[year][columns].sum(axis = 1)
    
    totcalls[year] = tract_count[year]['Total'].sum()    

totcalls

{2004: 2842831.0,
 2005: 3166898.0,
 2006: 3050280.0,
 2007: 3131375.0,
 2008: 3118685.0,
 2009: 3122964.0,
 2010: 3137261.0,
 2011: 3116666.0,
 2012: 3250905.0,
 2013: 3265444.0,
 2014: 3331774.0,
 2015: 3379166.0,
 2016: 3346859.0}

In [621]:
tract_count[2004].head()

,APPLIANCE,CONSTRUCTION,ELECTRIC,GENERAL CONSTRUCTION,HEATING,NONCONST,PAINT - PLASTER,PLUMBING,Tract,Year
0,0.0,0.0,3.0,9.0,79.0,21.0,22.0,36.0,1,2004
1,1.0,0.0,12.0,8.0,26.0,23.0,4.0,4.0,10,2004
2,0.0,0.0,0.0,0.0,8.0,0.0,0.0,3.0,10.01,2004
3,3.0,0.0,16.0,45.0,92.0,41.0,46.0,37.0,100,2004
4,5.0,1.0,2.0,5.0,32.0,5.0,4.0,15.0,1004,2004


array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])